# 1) 데이터 준비와 전처리

In [113]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [114]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [115]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [116]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(20)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


# 2) 분석해 봅시다.

In [117]:
# ratings에 있는 유니크한 영화 개수

ratings['movie_id'].nunique()

3628

In [118]:
# ratings에 있는 유니크한 사용자 수

ratings['user_id'].nunique()

6039

In [119]:
# movies와 ratings를 합쳐준다.

data= pd.merge(movies, ratings)

In [120]:
# 사용하는 컬럼만 남겨줍니다.

using_cols = [ 'user_id','title', 'rating']
data = data[using_cols]
data.head(10)

,user_id,title,rating
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
5,18,Toy Story (1995),4
6,19,Toy Story (1995),5
7,21,Toy Story (1995),3
8,23,Toy Story (1995),4
9,26,Toy Story (1995),3


In [121]:
data['title'] = data['title'].str.lower() 

In [122]:
#가장 인기있는 영화 30개(인기순)

movies_count = data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

# 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.

In [123]:
# 내가 좋아하시는 영화의 데이터로 바꿔서 추가한다.
my_favorite = ['men in black (1997)' , 'fugitive, the (1993)' ,'terminator, the (1984)' ,
               'back to the future (1985)' ,'forrest gump (1994)']

# 'kimin'이라는 user_id가 위 영화의 평점을 5점씩 주었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['kimin']*5, 'title': my_favorite, 'rating':[5]*5})

# user_id에 'kimin'이라는 데이터가 없다면
# 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 
if not data.isin({'user_id':['kimin']})['user_id'].any():  
    data = data.append(my_playlist)   
    
data.tail(20) 

,user_id,title,rating
836463,5074,"contender, the (2000)",4
836464,5087,"contender, the (2000)",4
836465,5100,"contender, the (2000)",4
836466,5205,"contender, the (2000)",4
836467,5304,"contender, the (2000)",4
836468,5333,"contender, the (2000)",4
836469,5359,"contender, the (2000)",5
836470,5405,"contender, the (2000)",4
836471,5475,"contender, the (2000)",5
836472,5602,"contender, the (2000)",3


In [124]:
# 고유한 유저, 타이틀을 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 타이틀 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [125]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 

print(user_to_idx['kimin'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039이 나와야 합니다. 
print(title_to_idx['forrest gump (1994)'])

6039
342


In [126]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# temp_title_data을 통해 title 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')
    

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,title,rating
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
0,6039,1419,5
1,6039,439,5
2,6039,1122,5
3,6039,1152,5


# 4) CSR matrix를 직접 만들어 봅시다.

In [127]:
data['rating']

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64

In [130]:
print(data.rating)

0    5
1    4
2    4
3    5
4    5
    ..
0    5
1    5
2    5
3    5
4    5
Name: rating, Length: 836483, dtype: int64


In [131]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.rating, (data.user_id, data.title)), shape=(num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [132]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [133]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [134]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [135]:
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

In [136]:
kimin, back_to_the_future = user_to_idx['kimin'], title_to_idx['back to the future (1985)']
kimin_vector, back_to_the_future_vector = als_model.user_factors[kimin], als_model.item_factors[back_to_the_future]

In [137]:
kimin_vector

array([-1.08668752e-01,  3.17691833e-01, -1.51047617e-01, -5.24647720e-02,
       -5.79231009e-02, -3.16968977e-01, -1.44117087e-01,  1.99608207e-01,
       -2.25240082e-01,  8.88806954e-02,  1.32942185e-01, -6.25611246e-01,
        2.61385143e-01, -1.46868616e-01,  2.66936809e-01,  6.71692550e-01,
       -2.26440504e-02,  2.05191046e-01, -1.57286748e-01, -3.96014005e-02,
       -6.99364394e-02, -1.31861687e-01, -1.48717332e+00,  4.63588610e-02,
        2.93862283e-01,  1.91342473e-01,  6.85647875e-02, -3.10225904e-01,
       -3.90576869e-01,  2.90648401e-01,  6.66907430e-01,  9.66069460e-01,
        1.62810773e-01, -8.72056365e-01, -2.07042217e-01, -1.58967108e-01,
        1.99261621e-01, -2.19236583e-01,  1.04094946e+00, -1.00442991e-01,
       -5.34852326e-01, -4.68394101e-01,  7.48043656e-01, -5.21538317e-01,
       -2.55321831e-01,  6.84327260e-03,  8.34028602e-01, -1.88768983e-01,
        2.67478637e-04,  2.49831989e-01, -2.86177307e-01,  1.46652550e-01,
        5.80478132e-01, -

In [138]:
back_to_the_future_vector

array([-0.01550544,  0.03292141, -0.01431361,  0.00563815, -0.00873796,
        0.00485824, -0.01692422, -0.00684364,  0.00775661,  0.02451331,
        0.00269881, -0.02098045, -0.00778864,  0.0234304 , -0.0033843 ,
        0.05401371,  0.05114716, -0.015971  , -0.01462333, -0.01723523,
       -0.00808632,  0.01153312, -0.04722116,  0.00046202,  0.0001321 ,
        0.00335895, -0.01567796, -0.02193001,  0.01695528,  0.03932969,
        0.02661522,  0.03015262,  0.01142733, -0.00592729,  0.00358481,
        0.02576964,  0.00812993,  0.0040745 ,  0.01302521,  0.01633984,
       -0.01857228, -0.01343484,  0.02836139, -0.00290284, -0.01779318,
        0.01329921,  0.02515914, -0.02668072,  0.03803848,  0.02963958,
       -0.01446177,  0.02748494,  0.03990498,  0.00248421, -0.00963519,
        0.01391919,  0.01805923, -0.00526107, -0.02166562,  0.01214846,
       -0.02064386,  0.01544867,  0.06521616, -0.0113533 ,  0.04936884,
       -0.00779443,  0.03034828, -0.03476865,  0.02856064,  0.00

# 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [139]:
np.dot(kimin_vector, back_to_the_future_vector)

0.588475

In [140]:
toy = title_to_idx['toy story (1995)']
toy_vector = als_model.item_factors[toy]
np.dot(kimin_vector, toy_vector)

0.23137257

# 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [141]:
idx_to_title = {v:k for k,v in title_to_idx.items()}

def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

In [142]:
get_similar_title('men in black (1997)') 

['men in black (1997)',
 'jurassic park (1993)',
 'terminator 2: judgment day (1991)',
 'total recall (1990)',
 'independence day (id4) (1996)',
 'fifth element, the (1997)',
 'matrix, the (1999)',
 'lost world: jurassic park, the (1997)',
 'face/off (1997)',
 'rocky horror picture show, the (1975)']

In [143]:
get_similar_title('toy story (1995)') 

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'aladdin (1992)',
 'babe (1995)',
 'groundhog day (1993)',
 'lion king, the (1994)',
 'pleasantville (1998)',
 'beauty and the beast (1991)',
 "there's something about mary (1998)"]

In [144]:
get_similar_title('back to the future (1985)') 

['back to the future (1985)',
 "ferris bueller's day off (1986)",
 'back to the future part ii (1989)',
 'when harry met sally... (1989)',
 'ghostbusters (1984)',
 'big (1988)',
 'bull durham (1988)',
 'e.t. the extra-terrestrial (1982)',
 'fish called wanda, a (1988)',
 'airplane! (1980)']

# 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [145]:
user = user_to_idx['kimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
for i in title_recommended: 
    print("{} {:08.3f}".format(idx_to_title[i[0]], i[1]))

terminator 2: judgment day (1991) 0000.620
matrix, the (1999) 0000.558
jurassic park (1993) 0000.546
total recall (1990) 0000.416
star wars: episode vi - return of the jedi (1983) 0000.342
hunt for red october, the (1990) 0000.322
braveheart (1995) 0000.320
die hard (1988) 0000.315
saving private ryan (1998) 0000.313
star wars: episode iv - a new hope (1977) 0000.304
star wars: episode v - the empire strikes back (1980) 0000.297
e.t. the extra-terrestrial (1982) 0000.297
groundhog day (1993) 0000.292
speed (1994) 0000.275
sixth sense, the (1999) 0000.274
airplane! (1980) 0000.261
aliens (1986) 0000.235
toy story (1995) 0000.231
independence day (id4) (1996) 0000.222
alien (1979) 0000.212


In [146]:
rihanna = title_to_idx['die hard (1988)']
explain = als_model.explain(user, csr_data, itemid=rihanna)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('terminator, the (1984)', 0.21170626266454695),
 ('fugitive, the (1993)', 0.16700344544559192),
 ('back to the future (1985)', 0.02882417767006322),
 ('forrest gump (1994)', -0.019415525598657744),
 ('men in black (1997)', -0.07613123084666552)]

## 회고록

추천 시스템의 개념 이해는 어려웠다. 우선 노드를 따라서 참고해서 작성을 해보고 오류가 나는 부분은 다른 블러거 글들을 참고 하면서 작성하였다.
rationg과 rating에서 오타로 인해 계속 오류가 나서 시간이 많이 걸렸다. 
나의 추천 영화 선호도 예측률은 많이 낮았다. 그게 좀 아쉬웠다. 